In [1]:
from dotenv import load_dotenv
import os

load_dotenv("/teamspace/studios/this_studio/beta/.env")


True

In [2]:
import sys
import os

# Get the current working directory
current_dir = os.getcwd()

# Add the parent directory to sys.path
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

In [1]:
from beta import Artifact

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json


def on_file_selected(change):
    if change['type'] == 'change' and change['name'] == 'value':
        file_content = change['new']
        if not file_content:
            print("No file selected")
            return
        
        print("File content type:", type(file_content))
        print("File content:", file_content)
        
        # Handle the tuple structure
        if isinstance(file_content, tuple) and len(file_content) > 0:
            file_data = file_content[0]
            file_name = file_data['name']
            
            print(f"Selected file: {file_name}")
            print("File data type:", type(file_data))
            
            try:
                # Create the Artifact
                artifact = Artifact(files=[file_name])
                print("Artifact created successfully")
                print("Artifact DataFrame:")
                artifact.df.collect().show()
            except Exception as e:
                print(f"Error creating Artifact: {str(e)}")
        else:
            print("Unexpected file content format")

# Create a file upload widget
file_upload = widgets.FileUpload(
    accept='',  # Accept all file types
    multiple=False  # Allow only single file selection
)

# Display the file upload widget
display(file_upload)

# Attach the callback to the file upload widget
file_upload.observe(on_file_selected, names='value')

In [2]:
papers = Artifact(files=[
    "/teamspace/studios/this_studio/beta/examples/data/Papers/2401.05459v2.pdf",
    "/teamspace/studios/this_studio/beta/examples/data/Papers/2401.18059.pdf",
    "/teamspace/studios/this_studio/beta/examples/data/Papers/2402.01680v2.pdf",
    "/teamspace/studios/this_studio/beta/examples/data/Papers/2402.01968v1.pdf"
])


INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0


In [3]:
papers.df.collect()

INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: PyMicroPartition::to_table, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: MicroPartition::concat, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0
INFO:daft_io.stats:IOStatsContext: PyMicroPartition::to_table, Gets: 0, Heads: 0, Lists: 0, BytesRead: 0, AvgGetSize: 0, BytesUploaded: 0, AvgPutSize: 0


artifact_uriUtf8,checksumUtf8,created_atUtf8,extensionUtf8,idUtf8,inserted_atUtf8,mime_typeUtf8,nameUtf8,payloadBinary,size_bytesInt64,typeUtf8,updated_atUtf8,versionUtf8
,9446b09e3c50abf6b0bc1fcec3f97e09,2024-09-20T16:41:19.589546,pdf,01J884YDF5W3ZRF8KZ9X90KN7M,2024-09-20T16:41:19.589546,application/pdf,2401.05459v2.pdf,"b""\x1f\x8b\x08\x00\xaf\xa5\xedf\x02\xff""...",4276555,file,2024-09-20T16:41:19.589546,1.0
,bef90ad5efc208f8c214dec982b3f4a1,2024-09-20T16:41:19.812750,pdf,01J884YDP4M1YQXQQA9M4RZFM7,2024-09-20T16:41:19.812750,application/pdf,2401.18059.pdf,"b""\x1f\x8b\x08\x00\xaf\xa5\xedf\x02\xff""...",2547113,file,2024-09-20T16:41:19.812750,1.0
,6ebd968a706c836ef2a9062d59c6bb7c,2024-09-20T16:41:19.897665,pdf,01J884YDRSYMCEVD7CEQ9GA264,2024-09-20T16:41:19.897665,application/pdf,2402.01680v2.pdf,"b""\x1f\x8b\x08\x00\xaf\xa5\xedf\x02\xff""...",1243493,file,2024-09-20T16:41:19.897665,1.0
,f4e5524a0215616816ec4716c6bf709f,2024-09-20T16:41:19.944882,pdf,01J884YDT87DQXB6YS96VHVQW0,2024-09-20T16:41:19.944882,application/pdf,2402.01968v1.pdf,"b""\x1f\x8b\x08\x00\xaf\xa5\xedf\x02\xff""...",418187,file,2024-09-20T16:41:19.944882,1.0


In [ ]:
from beta.agents import Agent
from beta.models.obj.model_object import VLLM
from vllm import LLM
from beta.tools.obj import CalculatorTool, DataFrameTool
from beta.models.serve.engines import VLLMEngine
from beta.models.serve.engines import OpenAIEngineConfig
from mlflow.client import MlflowClient
from daft import DataFrame
from beta.models.serve.engines.openai_engine import OpenAIEngine

In [ ]:
!pip install mlflow


In [ ]:
import mlflow
mlflow.set_experiment("Dratos AI")
with mlflow.start_run():
    mlflow.log_param("model", "gpt-4o")
    mlflow.log_param("temperature", 0.5)
    mlflow.log_param("max_tokens", 4096)
    mlflow.log_param("top_p", 1)
    mlflow.log_param("frequency_penalty", 0)
    mlflow.log_param("presence_penalty", 0)
    prompt = "What is the capital of Africa?"
    mlflow.log_param("prompt", prompt)

    config = OpenAIEngineConfig(data={
        "api_key": os.getenv("OPENAI_API_KEY"),
        "temperature": 0.5,
        "max_tokens": 4096,
        "top_p": 1,
        "frequency_penalty": 0,
        "presence_penalty": 0
    })
    engine = OpenAIEngine(model_name="gpt-4o", config=config)
    response = await engine.generate_structured(prompt, structure="{city: str, capital: str}")

print(response)


In [ ]:
from beta.prompts import prompt

@prompt
def my_prompt(question):
    """{{ question }}"""

@prompt
def cot(my_prompt, max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

In [ ]:
from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date
from ray import serve
@serve.deployment(name="Test0")
class QuarterlyEarnings(BaseModel):
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")

    def __call__(self, *args, **kwargs):
        return "QuarterlyEarnings deployment is working"


In [ ]:
import ray
from beta.models.serve.engines.openai_engine import OpenAIEngine

def openai_engine_serializer(engine):
    # Return the arguments needed to reconstruct the engine
    return (engine.model_name, engine.config)

def openai_engine_deserializer(model_name, config):
    # Reconstruct the engine
    return OpenAIEngine(model_name=model_name, config=config)

# Register the custom serializer with Ray
ray.util.register_serializer(
    OpenAIEngine,
    serializer=openai_engine_serializer,
    deserializer=openai_engine_deserializer
)

In [ ]:
from datetime import datetime
import inspect
from pydantic import BaseModel
import daft
from ray import serve
from typing import get_origin, get_args

def pydantic_to_daft_schema(model_class):
    if isinstance(model_class, serve.Deployment):
        # This is a Ray Serve deployment
        model_class = model_class.func_or_class

    if not inspect.isclass(model_class) or not issubclass(model_class, BaseModel):
        raise ValueError("Input must be a Pydantic model class or a Ray Serve deployment of a Pydantic model")

    field_dict = {}
    for name, field in model_class.model_fields.items():
        if field.annotation == str:
            field_dict[name] = daft.DataType.string()
        elif field.annotation == int:
            field_dict[name] = daft.DataType.int64()
        elif field.annotation == float:
            field_dict[name] = daft.DataType.float64()
        elif field.annotation == bool:
            field_dict[name] = daft.DataType.boolean()
        elif field.annotation == datetime.date:
            field_dict[name] = daft.DataType.date()
        elif get_origin(field.annotation) == list and get_args(field.annotation)[0] == str:
            field_dict[name] = daft.DataType.list(daft.DataType.string())
        else:
            field_dict[name] = daft.DataType.python()  # Fallback for complex types
    return daft.DataType.struct(field_dict)

In [ ]:
# New Cell: Generate Sample Revenue DataFrames

from datetime import date
from pydantic import BaseModel
import daft

# Define sample data for Q4 2024
sample_data = [
    {
        "company_name": "ABC Corp",
        "fiscal_quarter": 4,
        "fiscal_year": 2024,
        "revenue": 500000.0,
        "net_income": 120000.0,
        "earnings_per_share": 2.5,
        "report_date": date(2024, 10, 31),
        "analyst_expectations_met": True,
        "key_highlights": [
            "Increased sales in electronics",
            "Expanded to new markets"
        ]
    },
    {
        "company_name": "ABC Corp",
        "fiscal_quarter": 4,
        "fiscal_year": 2024,
        "revenue": 600000.0,
        "net_income": 150000.0,
        "earnings_per_share": 3.0,
        "report_date": date(2024, 11, 30),
        "analyst_expectations_met": False,
        "key_highlights": [
            "Supply chain disruptions",
            "Higher marketing expenses"
        ]
    },
    {
        "company_name": "ABC Corp",
        "fiscal_quarter": 4,
        "fiscal_year": 2024,
        "revenue": 550000.0,
        "net_income": 130000.0,
        "earnings_per_share": 2.8,
        "report_date": date(2024, 12, 31),
        "analyst_expectations_met": True,
        "key_highlights": [
            "Improved operational efficiencies",
            "New product launch success"
        ]
    }
]

# Convert sample data to QuarterlyEarnings instances
quarterly_earnings_instances = [QuarterlyEarnings(**item) for item in sample_data]

# Create a Daft DataFrame from Pydantic models
revenue_df = daft.from_pydantic(QuarterlyEarnings, quarterly_earnings_instances)

# Display the DataFrame
print("Sample Revenue DataFrame:")
revenue_df.collect().show()

In [ ]:
from daft import Schema
import daft
import json
import ray
from ray import serve
from beta.agents.agent import Agent, Metadata, SchemaWrapper
from beta.tools.obj.calculator_tool import CalculatorTool
from beta.tools.obj.dataframe_tool import DataFrameTool
from beta.models.serve.engines.openai_engine import OpenAIEngine, OpenAIEngineConfig

# Initialize Ray and Serve
ray.init(ignore_reinit_error=True)
serve.start()

# Create OpenAIEngine instances
config = OpenAIEngineConfig(data={
    "api_key": os.getenv("OPENAI_API_KEY"),
    "temperature": 0.5,
    "max_tokens": 4096,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0
})  
model_engine = OpenAIEngine(model_name="gpt-4o", config=config)
embedding_engine = OpenAIEngine(model_name="gpt-4o", config=config)
stt_engine = OpenAIEngine(model_name="gpt-4o", config=config)

# QuarterlyEarnings as a DataFrame Schema
QuarterlyEarningsSchema = pydantic_to_daft_schema(QuarterlyEarnings)

# Create the Agent
q1_agent = Agent(
    name="Q1 Agent",
    model=model_engine,
    embedding=embedding_engine,
    stt=stt_engine,
    tools=[CalculatorTool, DataFrameTool],
    metadata=Metadata(schema=SchemaWrapper(schema=QuarterlyEarningsSchema)),
    engine=model_engine,
    memory=[earnings],
    is_async=True
)

from beta.prompts import prompt

@prompt
def my_prompt(question):
    """What were the key highlights of the Q4 2024 earnings?"""

response = await q1_agent.process(prompt=my_prompt())
print(response)


In [ ]:
# New Cell: Using CalculatorTool and DataFrameTool with Agent

from beta.tools import CalculatorTool, DataFrameTool
from beta.agents import Agent
from beta.prompts import prompt

# Define a new prompt that leverages both tools
@prompt
def compute_and_analyze(question):
    """
    You are an intelligent assistant equipped with Calculator and DataFrame tools.
    
    Task:
    - Calculate the total revenue for Q4 2024.
    - Provide a summary of key highlights in a table format.
    
    User Question:
    {{ question }}
    """

# Initialize the Agent with CalculatorTool and DataFrameTool
compute_analyze_agent = Agent(
    name="ComputeAnalyzeAgent",
    model=model_engine,          # Ensure model_engine is defined in your environment
    embedding=embedding_engine,  # Ensure embedding_engine is defined in your environment
    stt=stt_engine,              # Ensure stt_engine is defined in your environment
    tools=[CalculatorTool, DataFrameTool],
    metadata=Metadata(schema=SchemaWrapper(schema=QuarterlyEarningsSchema)),
    engine=model_engine,
    is_async=True
)

# Define the question to be processed
question = "Calculate the total revenue for Q4 2024 and provide a summary of key highlights in a table."

# Debugging: Print the constructed prompt before processing
constructed_prompt = compute_and_analyze(question)
print("Constructed Prompt:")
print(constructed_prompt)

# Process the prompt with the agent
response = await compute_analyze_agent.process(prompt=constructed_prompt)

# Display the response
print("Agent Response:")
print(response)

In [ ]:
!pip install returns

In [ ]:
from returns.future import Future

async def process_agent_response(prompt):
    return Future.from_sync(lambda: q1_agent.process(my_prompt(prompt)))

response = await process_agent_response("What were the key highlights of the Q4 2024 earnings?").close()
print(response)

In [ ]:
mlflow_client = MlflowClient("http://localhost:5000")
config = OpenAIEngineConfig(data={"temperature": 0.5, "max_tokens": 4096, "top_p": 1, "frequency_penalty": 0, "presence_penalty": 0})
llm = LLM(model="meta-llama/Meta-Llama-3.1-8B-Instruct")
vllm = VLLM(model=LLM)
engine = VLLMEngine(model_name="meta-llama/Meta-Llama-3.1-8B-Instruct", mlflow_client=mlflow_client, config=config, api_key="EMPTY", base_url="http://localhost:8000")

paligemma = vllm(
    model_name="google/paligemma-3b-mix-448",
    engine=engine,
)

In [ ]:
from beta.models.obj import llm
from beta.models.serve.engines import LitServeEngine, OpenAIEngine, OpenRouterEngine, VLLMEngine


completion_settings = llm.settings(
    temperature=0.5,
    max_tokens=4096,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
)

# Defaults to OpenRouter
# Expects OPENROUTER_API_KEY to be defined in .env
gemini_free = llm(
    model_name="google/gemini-pro-1.5-flash",
    settings=completion_settings,
)

# If OPENAI_API_KEY, uses { OpenAi | AsyncOpenAI } clients instead of OpenRouter. 
gpt4o = llm(
    model_name="openai/o1-mini",
    settings=completion_settings,
)

# If ANTHROPIC_API_KEY, uses {Anthropic | AsyncAnthopic }
sonnet = llm(
    model_name="anthopic/claude-3.5-sonnet",
    settings = completion_settings,
)


# Specifying the Engine with OS Model
paligemma = llm(
    model_name="google/paligemma-3b-mix-448",
    engine=LitServeEngine(), 
    completion_settings=completion_settings
)



In [ ]:
from beta.tools import CalculatorTool, DataframeTool

In [ ]:

from beta.agents import BaseAgent, SpeechAgent

artifact_df = daft.DataFrame()
node_df = daft.DataFrame()
document_df = daft.DataFrame()






earnings = Data.Projects.get("Earnings Q4 2024")
knowledge_graph = my_project.Graphs.get("latest")


q1_agent = Agent(
    llm=paligemma,
    tools=[CalculatorTool, DataframeTool],
    context=[earnings],
    memory={'short':[knowl]},
    planning=
    rethinking=
)


@prompt
def my_prompt(question):
    """{{ question}}"""

@prompt
def cot(my_prompt,max_steps):
    """
    System: 
        Using Chain of Thought:
        max_steps = {{max_steps}}

        for step in max_steps:
            Reason about the request and append, "Therefore"
        
        Make a concluding response 

    User:
    {{ my_prompt }}
    """

from pydantic import BaseModel, Field
from typing import List, Optional
from datetime import date

class QuarterlyEarnings(BaseModel):
    """
    Represents quarterly earnings data for a company.
    """
    company_name: str = Field(..., description="Name of the company")
    fiscal_quarter: int = Field(..., ge=1, le=4, description="Fiscal quarter (1-4)")
    fiscal_year: int = Field(..., description="Fiscal year")
    revenue: float = Field(..., description="Total revenue for the quarter")
    net_income: float = Field(..., description="Net income for the quarter")
    earnings_per_share: float = Field(..., description="Earnings per share")
    report_date: date = Field(..., description="Date of the earnings report")
    analyst_expectations_met: Optional[bool] = Field(None, description="Whether analyst expectations were met")
    key_highlights: Optional[List[str]] = Field(None, description="Key highlights from the earnings report")


response = await my_agent(
    prompt=
    response_model=[Address], 
        reponse_model_strict = False
)